# Q4 Transfer Learning with Pre-trained CNN 
Use a pre-trained convolutional neural network (CNN) as a feature extractor and fine-
tune a classifier on a subset of the CIFAR-10 dataset. Set random seeds to 42. Follow
the configuration below:
- Load CIFAR-10 and normalize pixel values to [0,1]
- Use only the first 2000 training samples and first 500 test samples
- Load MobileNetV2 from tensorflow.keras.applications, with include top=False and weights=’imagenet’
- Freeze all layers of the pre-trained base
- Add a classifier on top:
- GlobalAveragePooling2D
- Dense layer with 128 neurons, ReLU activation
- Dropout: 0.2
- Output layer: 10 neurons with softmax
- Optimizer: Adam, learning rate = 0.001
- Loss: sparse categorical crossentropy
- epochs = 5, batch size = 32

Q4.1 Report the test accuracy of the model.

### Imports and seed for reproducibility

In [42]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader, Subset
import numpy as np
import random


seed = 42
torch.manual_seed(seed)
np.random.seed(seed)
random.seed(seed)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')


## Loading CIFAR-10 with subsets 

In [43]:
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0,), (1,))])

train_full = datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
test_full = datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)

train_ds = Subset(train_full, range(2000))
test_ds = Subset(test_full, range(500))

train_dl = DataLoader(train_ds, batch_size=32, shuffle=True)
test_dl = DataLoader(test_ds, batch_size=32)


### Load MobileNetV2 base
In the task it specifies using MobileNetV2 from tensorflow.keras.applications. But since i use PyTorch, the pre-trained model is loaded from torchvision.models instead:

In [44]:

mobilenet = models.mobilenet_v2(weights=models.MobileNet_V2_Weights.IMAGENET1K_V1)
mobilenet.features.requires_grad_(False)  # freeze all convolutional layers


# Replace classifier

mobilenet.classifier = nn.Sequential(
    nn.Linear(mobilenet.last_channel, 128),  # dense to 128
    nn.ReLU(),
    nn.Dropout(0.2),
    nn.Linear(128, 10)  # 10 neurons for output layers
)
mobilenet = mobilenet.to(device)

# Optimizer and loss function
optimizer = optim.Adam(mobilenet.parameters(), lr=0.001)
criterion = nn.CrossEntropyLoss()


# Training loop
epoch_counter = 0

def training_loop(epochs):
    global epoch_counter
    print(f"Training for {epochs} more epochs")
    total_epochs = epoch_counter + epochs
    for epoch in range(epochs):
        epoch_counter += 1
        mobilenet.train()
        running_loss = 0
        for x, y in train_dl:
            x, y = x.to(device), y.to(device)
            optimizer.zero_grad()
            out = mobilenet(x)
            loss = criterion(out, y)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
        print(f"Epoch {epoch+1}/{total_epochs}, Loss: {running_loss/len(train_dl):.4f}")



def evaluate():
    mobilenet.eval()
    correct, total = 0, 0
    with torch.no_grad():
        for x, y in test_dl:
            x, y = x.to(device), y.to(device)
            pred = mobilenet(x).argmax(1)
            correct += (pred == y).sum().item()
            total += y.size(0)
    print(f"Accuracy: {100*correct/total:.2f}%")





#for i in range(5):
#    training_loop(1)
#    evaluate()

training_loop(5)
evaluate()


Training for 5 more epochs
Epoch 1/5, Loss: 2.1091
Epoch 2/5, Loss: 1.8517
Epoch 3/5, Loss: 1.7231
Epoch 4/5, Loss: 1.6404
Epoch 5/5, Loss: 1.6360
Accuracy: 38.80%


MobileNetV2 already knows how to detect general image features like edges, colors, and textures from ImageNet and by freezing those layers and only training the new classifier, the model reuses this knowledge to recognize images from the dataset. Since only the classifier is trained on a small dataset (2,000 samples), the accuracy is lower. the model benefits from pretrained features but is limited by the small fine-tuning dataset and frozen layers.